In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport beanmachine.ppl

In [170]:
%autoreload

In [179]:
import beanmachine.ppl as bm
import torch
import torch.distributions as dist
import torch.optim as optim

from beanmachine.ppl.inference.variational.variational_infer import (
    kl_forward,
    kl_reverse
)

class NormalNormal:
    def __init__(self):
        self.device = 'cpu'

    @bm.random_variable
    def mu(self):
        return dist.Normal(
            torch.zeros(1).to(self.device), 10 * torch.ones(1).to(self.device)
        )

    @bm.random_variable
    def x(self, i):
        return dist.Normal(self.mu(), torch.ones(1).to(self.device))

model = NormalNormal()

@bm.param
def phi():
    return torch.zeros(2)  # mean, log std

@bm.random_variable
def q_mu():
    params = phi()
    return dist.Normal(params[0], params[1].exp())

for discrepancy in [kl_reverse, kl_forward]:
    world = bm.VariationalInfer().infer(
        queries_to_guides={model.mu(): q_mu()},
        observations={
            model.x(1): torch.tensor(3.0),
            model.x(2): torch.tensor(4.0),
        },
        num_steps=1000,
        num_samples=1,
        optimizer=lambda params: optim.Adam(params, lr=1e-1),
        discrepancy_fn=discrepancy,
    )
    print(world._run_node(q_mu())[0])
    print('-'*10)

  0%|          | 0/1000 [00:00<?, ?it/s]

Normal(loc: 3.60760498046875, scale: 0.7004327774047852)
----------


  0%|          | 0/1000 [00:00<?, ?it/s]

Normal(loc: 3.5967936515808105, scale: 1.1184630393981934)
----------
